<a href="https://colab.research.google.com/github/arcsin42/graph_visualization/blob/main/graph_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Клевцов Тимур, М05-013д  
В файлы ноутбука необходимо добавить граф в формате graphml или xml и прописать его имя в переменной name

In [91]:
import math
import plotly
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import networkx as nx
from itertools import *
#from google.colab import drive
#drive.mount('/content/drive')

Основной блок программы, содержащий все функции и алгоритмы. Ширина слоя W (-1 если не ограничена)

In [92]:
#name = "flow-test-1-loop-crush.xml"
name = "test_graph.xml"
W = 3 #ширина слоя
original = True #устанавливать ли настройки для примера из учебника 

infty = 1000000
L = []  #наши будущие слои
graph = []  #будущий граф
G = nx.read_graphml("./%s" % name)  #считаем граф из файла

#создадим класс для хранения координат вершины, ее потомков, родителей и метки
class my_node():
  label = infty  #означает непомеченную вершину
  x = 0.0
  y = 0.0
  childs = []
  parents = []
  def __init__(self, label=infty, x=0.0, y=0.0, childs=[], parents=[]):
    self.label = label
    self.x = x
    self.y = y
    self.childs = childs
    self.parents = parents

def get_parents_labels(node0: my_node): #получение списка меток родителей
  parents_labels = []
  for parent in node0.parents:
    parents_labels.append(graph[parent].label)
  return parents_labels

#лексикографическое сравнение массивов чисел
def lex_comparator(str1: [int], str2: [int]) -> bool: #true (>=) - первый больше второго, false (<) - наоборот
  l1 = len(str1)
  l2 = len(str2)
  l = min(l1, l2)
  if max(l1, l2) == 0:
    return True
  if l1 == 0:
    return False
  if l2 == 0:
    return True

  str1 = sorted(str1, reverse=True)
  str2 = sorted(str2, reverse=True)
  for i in range(l):
    if str1[i] > str2[i]:
      return True
    if str1[i] < str2[i]:
      return False
    if str1[i] == str2[i] and i == l - 1:
      if l2 > l1:
        return False
      else:
        return True
  return False

#нахождение минимальной по лексикографичности родителей вершины
def lex_min_node() -> int:
  unlable_graph = []
  for i in range(len(graph)):
    if graph[i].label == infty:
      unlable_graph.append(i)

  if len(unlable_graph) == 0:
    return -1
  
  parents_labels = [get_parents_labels(graph[i]) for i in unlable_graph]
  min_parents = parents_labels[0]
  min_index = 0
  for label in parents_labels:
    if lex_comparator(label, min_parents) == False: #то есть label < min_parents
      min_parents = label

  min_index = parents_labels.index(min_parents)
  return unlable_graph[min_index]


def label_graph():  #итоговая функция для пометки вершин
  for i in range(len(graph)):
   node_index = lex_min_node()
   graph[node_index].label = i + 1
  return

def find_unused_nodes() -> [int]:  #находим маскимальную по метке вершину с потомками в слоях до k
  nodes = []
  for i in range(len(graph)):
    if graph[i].y >= 1:
      continue

    childs = True
    for child in graph[i].childs:
      if graph[child].y < 1:  #проверяем, распределен ли потомок
        childs = False
    
    if childs:
      nodes.append(i)

  return nodes

def layered(w):  #распределение по слоям
  label_graph() #лексикографически отмечаем вершины
  L.append([])  #создаем первый слой
  queue = []
  start_nodes = find_unused_nodes()
  queue += start_nodes

  while len(queue) > 0:
    labels = [graph[i].label for i in queue]
    max_label = max(labels)
    current_node = queue[labels.index(max_label)]
    queue.remove(current_node)

    all_childs_lower = True
    for j in graph[current_node].childs:
      if graph[j].y >= len(L):
        all_childs_lower = False

    if len(L[-1]) < w and all_childs_lower:
      L[-1].append(current_node)
    else:
      l = []
      l.append(current_node)
      L.append(l)

    graph[current_node].y = len(L)
    if len(queue) == 0:
      queue += find_unused_nodes()
  return

def min_of_dummy(): #минимизация вспомогательных вершин
  label_graph() 
  L.append([]) 

  for i in range(len(graph)):
    start_nodes = find_unused_nodes()
    labels = [graph[i].label for i in start_nodes]
    max_label = max(labels)
    current_node = start_nodes[labels.index(max_label)]

    all_childs_lower = True
    for j in graph[current_node].childs:
      if graph[j].y >= len(L):
        all_childs_lower = False

    if all_childs_lower:
      L[-1].append(current_node)
    else:
      l = []
      l.append(current_node)
      L.append(l)

    graph[current_node].y = len(L)  #нумеровка слоев с 0, но координаты с 1

  return

def add_dummy():  #функция добавления вспомогательных вершин
  ll = len(graph)
  for i in range(ll):
    jj = len(graph[i].childs)
    for _j in range(jj): #i->j ребро
      j = graph[i].childs[_j]
      dy = graph[i].y - graph[j].y
      if abs(dy) >= 2 and graph[i].label >= 0 and graph[j].label >= 0:
        #print("edge", i, j)
        for k in range(1, dy):
          x = (float)(graph[i].x - graph[j].x) * k / (dy) + graph[j].x
          y = k + graph[j].y
          graph.append(my_node(-1, x, y, [],[]))
          l = len(graph) - 1
          L[y - 1].append(l)
          if k == dy - 1:
            graph[-1].parents.append(i)
            graph[i].childs[graph[i].childs.index(j)] = l
          if k == 1:
            graph[-1].childs.append(j)
            graph[j].parents[graph[j].parents.index(i)] = l
          if k > 1 and k < dy - 1:
            graph[-1].parents.append(l + 1)
            graph[-1].childs.append(l - 1)
          if k == 1 and dy > 2:
            graph[-1].parents.append(l + 1)
          if k == dy - 1 and dy > 2:
            graph[-1].childs.append(l - 1)
          print("dummy node was added ", l, i, j, x, y, graph[-1].childs, graph[-1].parents)
  return

def simple_position():  #функция наивного расчета x-координат
  for layer in L:
    for i in layer:
      graph[i].x = layer.index(i)
  return

def center_position():  #функция оцентровки графика
  max_size = max([len(layer) for layer in L])
  x_center = max_size / 2
  for layer in L:
    l_size = len(layer)
    for i in layer:
      graph[i].x = x_center + layer.index(i) - l_size / 2
  return

def number_of_crossing(level1, level2): #level1 < level2
  number = 0;
  for v1 in level2:
    for v2 in level2:
      if v1 == v2:
        continue
      for w1 in graph[v1].childs:
        if not w1 in level1:
          continue
        for w2 in graph[v2].childs:
          if not w2 in level1:
            continue
          cross = (level2.index(v1) - level2.index(v2)) * (level1.index(w1) - level1.index(w2))
          if cross < 0:
            number += 1

  return number // 2

def the_best_permutation(center:int): #функция лучший перестановки центрального слоя из трех
  result_perm = []
  min_crossing = infty
  for perm in permutations(L[center]):
    crossing = number_of_crossing(L[center - 1], perm) + number_of_crossing(perm, L[center + 1])
    #print(crossing, end=' ')
    if crossing < min_crossing:
      min_crossing = crossing
      result_perm = perm
    
  #print(min_crossing)
  return result_perm
"""
непосредственно выполнение программы начинается здесь

"""
nodes_dict = {}
number = 0

for node in G.nodes():  #пронумеровываем вершины из G, создаем для них словарь и расширяем graph
  nodes_dict[node] = number
  graph.append(my_node(infty, 0.0, 0.0, [],[]))
  number += 1

for edge in G.edges(): #разбираем ребра из G, добавляем их в структуру graph
  v0 = nodes_dict[edge[0]]
  v1 = nodes_dict[edge[1]]
  graph[v0].childs.append(v1)
  graph[v1].parents.append(v0)



if W > 0 or original:
  layered(W)  #алгоритм Грехема для слоев
else:
  min_of_dummy()  #если W не задана, применяем минимизацию вспомогательных вершин

if original:
  simple_position()


add_dummy() #добавление вспомогательных вершин с расчетом их координат
if not original:
  simple_position()

if original:
  graph[18].x += 1

for i in range(len(L) - 1, -1, -1): #вывод распределений по слоям
  print(L[i])
#for i in range(len(graph)):
#  print(i, graph[i].childs, graph[i].parents, graph[i].x, graph[i].y)

#print(number_of_crossing(L[2],L[3]))
if not original:
  for i in range(10):
    for level in range(len(L) - 2, 0, -1):
      L[level] = the_best_permutation(level)
# simple_position() #здесь можно выбрать тип распределения по оси х внутри каждого слоя, либо слева направо, либо по центру
  center_position()

dummy node was added  13 3 10 0.5 4 [10] [3]
dummy node was added  14 4 6 0.75 2 [6] [15]
dummy node was added  15 4 6 0.5 3 [14] [16]
dummy node was added  16 4 6 0.25 4 [15] [4]
dummy node was added  17 4 9 0.5 4 [9] [4]
dummy node was added  18 4 7 1.0 4 [7] [4]
[1]
[4, 3]
[8, 5, 13, 16, 17, 18]
[10, 9, 7, 15]
[12, 11, 14]
[0, 6, 2]


Блок отрисовки, метки вершин показываются при наведении. Ребра в виде отрезков, но, посколько они всегда идут сверху вниз, надеюсь, это не критично.

In [93]:
fig = go.Figure()

for node in graph:  #добавление ребер на холст
  for node_2 in node.childs:
    v0 = node
    v1 = graph[node_2]
    fig.add_trace(go.Scatter(
        x=[v0.x, v1.x], 
        y=[v0.y, v1.y], 
        line=dict(color='green', width=1)
        ))

def color(label):
  if label >= 0:
    return 'black'
  else:
    return 'green'

for node in graph:  #добавление вершин на холст
  fig.add_trace(go.Scatter(
     x = [node.x], 
     y = [node.y], 
     mode = 'markers+text',
     marker=dict(size=[15], color=color(node.label)),
     #text = str(graph.index(node))
     text = str(node.label)
     ))

#задание характеристик холста
fig.update_layout(  
    autosize=False,
    width=500,
    height=700,
    margin=dict(l=50, r=50, b=50, t=50, pad=4),
    paper_bgcolor="LightSteelBlue",
    showlegend = False
)
fig.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
